In [1]:
import numpy as np 
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.model_selection import GroupKFold, StratifiedKFold, train_test_split
from sklearn import metrics
from tqdm import tqdm
from scipy import signal
from tsfresh.feature_extraction import feature_calculators
from pykalman import KalmanFilter
from scipy.stats import percentileofscore
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 10000)

In [2]:
trn = pd.read_pickle('../input/feats_tblr/trn_dat_orig_v2_all.pkl')

In [3]:
list(trn.columns)

['grad_1',
 'grad_2',
 'grad_3',
 'grad_4',
 'lowpass_lf_0.0100',
 'lowpass_ff_0.0100',
 'lowpass_lf_0.0154',
 'lowpass_ff_0.0154',
 'lowpass_lf_0.0239',
 'lowpass_ff_0.0239',
 'lowpass_lf_0.0369',
 'lowpass_ff_0.0369',
 'lowpass_lf_0.0570',
 'lowpass_ff_0.0570',
 'lowpass_lf_0.0880',
 'lowpass_ff_0.0880',
 'lowpass_lf_0.1359',
 'lowpass_ff_0.1359',
 'lowpass_lf_0.2100',
 'lowpass_ff_0.2100',
 'lowpass_lf_0.3244',
 'lowpass_ff_0.3244',
 'lowpass_lf_0.5012',
 'lowpass_ff_0.5012',
 'highpass_lf_0.0100',
 'highpass_ff_0.0100',
 'highpass_lf_0.0163',
 'highpass_ff_0.0163',
 'highpass_lf_0.0264',
 'highpass_ff_0.0264',
 'highpass_lf_0.0430',
 'highpass_ff_0.0430',
 'highpass_lf_0.0699',
 'highpass_ff_0.0699',
 'highpass_lf_0.1136',
 'highpass_ff_0.1136',
 'highpass_lf_0.1848',
 'highpass_ff_0.1848',
 'highpass_lf_0.3005',
 'highpass_ff_0.3005',
 'highpass_lf_0.4885',
 'highpass_ff_0.4885',
 'highpass_lf_0.7943',
 'highpass_ff_0.7943',
 'ewm_mean_10',
 'ewm_std_10',
 'ewm_mean_50',
 'ewm_std

In [2]:
def read_data():
    print('Reading training, testing and submission data...')
    
    train = pd.read_csv('../input/train_clean.csv')
    
    test = pd.read_csv('../input/test_clean.csv')
    
    submission = pd.read_csv('../input/sample_submission.csv', dtype={'time':str})
    print('Train set has {} rows and {} columns'.format(train.shape[0], train.shape[1]))
    print('Test set has {} rows and {} columns'.format(test.shape[0], test.shape[1]))
    return train, test, submission

def get_batch(train, test):
    # concatenate data
    batch = 10
    total_batches = 14*5
    train['set'] = 'train'
    test['set'] = 'test'
    data = pd.concat([train, test])
    
    for i in range(int(total_batches)):
        data.loc[(data['time'] > i * batch) & (data['time'] <= (i + 1) * batch), 'batch'] = i + 1
        
    train = data[data['set'] == 'train']
    test = data[data['set'] == 'test']
    train.drop(['set'], inplace = True, axis = 1)
    test.drop(['set'], inplace = True, axis = 1)
    del data
    return train, test

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        if col!='open_channels':
            col_type = df[col].dtypes
            if col_type in numerics:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def calc_gradients(s, n_grads = 4):
    '''
    Calculate gradients for a pandas series. Returns the same number of samples
    '''
    grads = pd.DataFrame()
    
    g = s.values
    for i in range(n_grads):
        g = np.gradient(g)
        grads['grad_' + str(i+1)] = g
        
    return grads

def calc_low_pass(s, n_filts=10):
    '''
    Applies low pass filters to the signal. Left delayed and no delayed
    '''
    wns = np.logspace(-2, -0.3, n_filts)
    
    low_pass = pd.DataFrame()
    x = s.values
    for wn in wns:
        b, a = signal.butter(1, Wn=wn, btype='low')
        zi = signal.lfilter_zi(b, a)
        low_pass['lowpass_lf_' + str('%.4f' %wn)] = signal.lfilter(b, a, x, zi=zi*x[0])[0]
        low_pass['lowpass_ff_' + str('%.4f' %wn)] = signal.filtfilt(b, a, x)
        
    return low_pass

def calc_high_pass(s, n_filts=10):
    '''
    Applies high pass filters to the signal. Left delayed and no delayed
    '''
    wns = np.logspace(-2, -0.1, n_filts)
    
    high_pass = pd.DataFrame()
    x = s.values
    for wn in wns:
        b, a = signal.butter(1, Wn=wn, btype='high')
        zi = signal.lfilter_zi(b, a)
        high_pass['highpass_lf_' + str('%.4f' %wn)] = signal.lfilter(b, a, x, zi=zi*x[0])[0]
        high_pass['highpass_ff_' + str('%.4f' %wn)] = signal.filtfilt(b, a, x)
        
    return high_pass

def calc_ewm(s, windows=[50, 100, 200, 500, 1000]):
    '''
    Calculates exponential weighted functions
    '''
    ewm = pd.DataFrame()
    for w in windows:
        ewm['ewm_mean_' + str(w)] = s.ewm(span=w, min_periods=1).mean()
        ewm['ewm_std_' + str(w)] = s.ewm(span=w, min_periods=1).std()
        
    # add zeros when na values (std)
    ewm = ewm.fillna(value=0)
        
    return ewm


def add_features(s):
    '''
    All calculations together
    '''
    
    gradients = calc_gradients(s)
    low_pass = calc_low_pass(s)
    high_pass = calc_high_pass(s)
    ewm = calc_ewm(s)
    
    return pd.concat([s, gradients, low_pass, high_pass, ewm], axis=1)
#     return pd.concat([s, gradients, ewm], axis=1)


def divide_and_add_features(s, signal_size=100000):
    '''
    Divide the signal in bags of "signal_size".
    Normalize the data dividing it by 15.0
    '''
    # normalize
    s = s / 15.0
    
    ls = []
    # this is just to divide the data up into batches (bags) to keep track of progress
    # output is still the same size as input
    for i in tqdm(range(int(s.shape[0]/signal_size))):
        sig = s[i*signal_size:(i+1)*signal_size].copy().reset_index(drop=True)
        sig_featured = add_features(sig)
        ls.append(sig_featured)
    
    return pd.concat(ls, axis=0)

def rolling_features(train, test):
    
    pre_train = train.copy()
    pre_test = test.copy()
    
        
    for df in [pre_train, pre_test]:
        
        for window in tqdm([10, 50, 100, 200]):
            # roll backwards
            df['neighbour_quantile_' + str(window)] = df.groupby(['batch'])['signal'].transform(
                lambda x: x.rolling(window).apply(lambda x: percentileofscore())
            )
            
    del train, test, min_max
    
    return pre_train, pre_test

def Kalman1D(observations, damping=1):
    # To return the smoothed time series data
    observation_covariance = damping
    initial_value_guess = observations[0]
    transition_matrix = 1
    transition_covariance = 0.1
    kf = KalmanFilter(
            initial_state_mean=initial_value_guess,
            initial_state_covariance=observation_covariance,
            observation_covariance=observation_covariance,
            transition_covariance=transition_covariance,
            transition_matrices=transition_matrix
        )
    pred_state, state_cov = kf.smooth(observations)
    return pred_state

In [3]:
train, test, submission = read_data()

Reading training, testing and submission data...
Train set has 5000000 rows and 3 columns
Test set has 2000000 rows and 2 columns


In [6]:
train['batch'] = train.index // 100000
test['batch'] = test.index // 100000

In [13]:
train

,time,signal,open_channels,batch
0,0.0001,-2.760000,0,0
1,0.0002,-2.855700,0,0
2,0.0003,-2.407400,0,0
3,0.0004,-3.140400,0,0
4,0.0005,-3.152500,0,0
...,...,...,...,...
4999995,499.9996,2.919274,7,49
4999996,499.9997,2.697906,7,49
4999997,499.9998,4.516337,8,49
4999998,499.9999,5.639669,9,49


In [18]:
train['kal_signal'] = train.groupby('batch')['signal'].transform(lambda x: Kalman1D(x.values).squeeze())

In [17]:
test['kal_signal'] = test.groupby('batch')['signal'].transform(lambda x: Kalman1D(x.values).squeeze())

In [23]:
train['kal_diff'] = train['kal_signal'] - train['signal']
train['kal_ratio'] = train['kal_diff'] / train['signal']
test['kal_diff'] = test['kal_signal'] - test['signal']
test['kal_ratio'] = test['kal_diff'] / test['signal']

In [28]:
import bloscpack as bp

In [29]:
bp.pack_ndarray_to_file(train.loc[:, ['kal_signal', 'kal_diff', 'kal_ratio']].values, '../input/trn_dat_kal_all.bp')

In [30]:
bp.pack_ndarray_to_file(test.loc[:, ['kal_signal', 'kal_diff', 'kal_ratio']].values, '../input/tst_dat_kal_all.bp')

In [9]:
pre_train0 = divide_and_add_features(train['signal'])
pre_test0 = divide_and_add_features(test['signal'])

pre_train0.reset_index(inplace=True, drop=True)
pre_test0.reset_index(inplace=True, drop=True)

pre_train0 = reduce_mem_usage(pre_train0)
pre_test0 = reduce_mem_usage(pre_test0)

100%|██████████| 20/20 [00:01<00:00, 10.61it/s]


Mem. usage decreased to 543.59 Mb (75.0% reduction)
Mem. usage decreased to 217.44 Mb (75.0% reduction)


In [21]:
train, test = get_batch(train, test)
pre_train1, pre_test1 = rolling_features(train, test)
pre_train1 = reduce_mem_usage(pre_train1)
pre_test1 = reduce_mem_usage(pre_test1)


100%|██████████| 6/6 [03:37<00:00, 36.19s/it]

100%|██████████| 6/6 [01:23<00:00, 13.89s/it]


Mem. usage decreased to 1926.42 Mb (60.5% reduction)
Mem. usage decreased to 770.57 Mb (60.5% reduction)


In [38]:
pre_train = pd.concat([pre_train0.drop(columns=['diff', 'batch']), pre_train1.drop(columns=['signal', 'batch', 'open_channels', 'time'])], axis = 1)
pre_test = pd.concat([pre_test0.drop(columns=['diff', 'batch']), pre_test1.drop(columns=['signal', 'batch', 'open_channels', 'time'])], axis = 1)

In [39]:
np.all([a == b for a, b in zip(pre_train.columns, pre_test)])

True

In [21]:
pre_train_lbl = pre_train.loc[:, ['open_channels']].copy()

In [22]:
pre_train_lbl.to_pickle('../input/trn_lbl_orig_refresh0_all.pkl')

In [40]:
pre_train.to_pickle('../input/trn_dat_refresh0_all.pkl')

In [41]:
pre_test.to_pickle('../input/tst_dat_refresh0_all.pkl')

In [9]:
features = [col for col in pre_train.columns if col not in ['open_channels', 'time', 'batch']]

In [7]:
import bloscpack as bp

In [12]:
bp.pack_ndarray_to_file(pre_train[features].values, '../input/trn_dat_orig_all.bp')
bp.pack_ndarray_to_file(pre_train[target].values, '../input/trn_lbl_orig_all.bp')

In [8]:
# params = {
#     'boosting_type': 'gbdt',
#     'metric': 'rmse',
#     'objective': 'regression',
#     'n_jobs': 6,
#     'seed': 236,
#     'num_leaves': 280,
#     'learning_rate': 0.026623466966581126,
#     'max_depth': 80,
#     'lambda_l1': 2.959759088169741,
#     'lambda_l2': 1.331172832164913,
#     'bagging_fraction': 0.9655406551472153,
#     'bagging_freq': 9,
#     'colsample_bytree': 0.6867118652742716
# }

params = {
    "boosting": "gbdt",
    "metric": 'rmse',
    'objective': 'huber',
    'random_state': 236,
    'num_leaves': 280,
    'learning_rate': 0.026623466966581126,
    'max_depth': 80,
    'reg_alpha': 2.959759088169741, # L1
    'reg_lambda': 1.331172832164913, # L2
    "bagging_fraction": 0.9655406551472153,
    "bagging_freq": 9,
    'colsample_bytree': 0.6867118652742716
}

In [11]:
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
target = 'open_channels'

In [11]:
for fold, (trn_ndcs, vld_ndcs) in enumerate(kf.split(pre_train, pre_train[target])):
    x_trn, x_vld = pre_train[features].iloc[trn_ndcs], pre_train[features].iloc[vld_ndcs]
    y_trn, y_vld = pre_train[target][trn_ndcs], pre_train[target][vld_ndcs]
    #trn_set = lgb.Dataset(x_trn, y_trn)
    #vld_set = lgb.Dataset(x_vld, y_vld)
    break

In [12]:
# model = lgb.train(params, trn_set, num_boost_round=10000, early_stopping_rounds=100, valid_sets=[vld_set], verbose_eval=50)
model = lgb.LGBMRegressor(**params, n_estimators=10000, n_jobs=12)
model.fit(X=x_trn, y=y_trn, eval_set=[(x_vld, y_vld)], eval_metric='rmse', verbose=50, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[50]	valid_0's rmse: 1.7831
[100]	valid_0's rmse: 1.13112
[150]	valid_0's rmse: 0.692472
[200]	valid_0's rmse: 0.377599
[250]	valid_0's rmse: 0.205497
[300]	valid_0's rmse: 0.159849
[350]	valid_0's rmse: 0.155435
[400]	valid_0's rmse: 0.155023
[450]	valid_0's rmse: 0.154923
[500]	valid_0's rmse: 0.154859
[550]	valid_0's rmse: 0.154815
[600]	valid_0's rmse: 0.154777
[650]	valid_0's rmse: 0.154733
[700]	valid_0's rmse: 0.154707
[750]	valid_0's rmse: 0.154669
[800]	valid_0's rmse: 0.154642
[850]	valid_0's rmse: 0.154608
[900]	valid_0's rmse: 0.15458
[950]	valid_0's rmse: 0.154569
[1000]	valid_0's rmse: 0.154553
[1050]	valid_0's rmse: 0.154533
[1100]	valid_0's rmse: 0.154519
[1150]	valid_0's rmse: 0.154485
[1200]	valid_0's rmse: 0.15447
[1250]	valid_0's rmse: 0.154459
[1300]	valid_0's rmse: 0.15445
[1350]	valid_0's rmse: 0.154436
[1400]	valid_0's rmse: 0.154421
[1450]	valid_0's rmse: 0.154407
[1500]	valid_0's rmse: 0.154396
[155

LGBMRegressor(bagging_fraction=0.9655406551472153, bagging_freq=9,
              boosting='gbdt', boosting_type='gbdt', class_weight=None,
              colsample_bytree=0.6867118652742716, importance_type='split',
              learning_rate=0.026623466966581126, max_depth=80, metric='rmse',
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=10000, n_jobs=12, num_leaves=280, objective='huber',
              random_state=236, reg_alpha=2.959759088169741,
              reg_lambda=1.331172832164913, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [13]:
vld_pred = model.predict(x_vld, num_iteration=model.best_iteration_)
vld_pred = np.round(np.clip(vld_pred, 0, 10)).astype(int)

In [14]:
f1 = metrics.f1_score(y_vld.astype(int), vld_pred, average = 'macro')
print(f1)

0.9384238806036712


In [47]:
pre_train[features].columns

Index(['signal', 'signal_clean', 'signal_clean', 'lag_t1', 'lag_t2', 'lag_t3',
       'lead_t1', 'lead_t2', 'lead_t3', 'signalmean_t1000',
       ...
       'ewm_mean_10', 'ewm_std_10', 'ewm_mean_50', 'ewm_std_50',
       'ewm_mean_100', 'ewm_std_100', 'ewm_mean_500', 'ewm_std_500',
       'ewm_mean_1000', 'ewm_std_1000'],
      dtype='object', length=209)

In [46]:
pre_train[features].columns[np.argsort(model.feature_importances_)]

Index(['abs_avgbatch_25000', 'signal_clean', 'abs_maxbatch_25000',
       'signal_clean', 'p25batch_25000', 'medianbatch_25000', 'p75batch_25000',
       'maxtominbatch_25000', 'rangebatch_25000', 'p90batch_25000',
       ...
       'highpass_lf_0.7943', 'highpass_ff_0.7943', 'grad_1', 'grad_3',
       'lag_t3', 'ewm_std_50', 'lead_t2', 'lead_t3', 'lead_t1', 'ewm_std_10'],
      dtype='object', length=209)

In [41]:
model.feature_importances_

(209,)

In [25]:
len(model.feature_importances_)

209

In [22]:
[i for i in model.feature_importances_ if i > 10000]

[11655, 10170, 10643, 12625]

In [23]:
np.array(features)[model.feature_importances_ > 10000]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 207 but corresponding boolean dimension is 209

In [24]:
len(features)

207